Steps in fine tuning a LLM on a custom dataset using QLoRA:

Setting up the NoteBook
Install required libraries
Loading dataset
Create Bitsandbytes configuration
Loading the Pre-Trained model
Tokenization
Test the Model with Zero Shot Inferencing
Pre-processing dataset
Preparing the model for QLoRA
Setup PEFT for Fine-Tuning
Train PEFT Adapter
Evaluate the Model Qualitatively (Human Evaluation)
Evaluate the Model Quantitatively (with ROUGE Metric)

In [ ]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login

interpreter_login()

import os
# disable Weights and Biases - we are not tracking our training metrics
os.environ['WANDB_DISABLED']="true"

In [ ]:
# Loading the dataset
huggingface_dataset_name = "neil-code/dialogsum-test"
dataset = load_dataset(huggingface_dataset_name)        # contains the fields dialogue, summary, topic and ID
dataset['train'][0]     

In [ ]:
# BitsAndBytes quantization configuration
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False

)

# Loading Phi-2 using 4-bit quantization from HuggingFace
model_name='microsoft/phi-2'
device_map={"": 0}
ORIGINAL_MODEL = AutoModelForCausalLM.from_retrained(
    model_name,
    device_map=device_map,              # Controls where the model is loaded (CPU/GPU/multiple GPUs)
    quantization_config=bnb_config,     # quantization configuration
    trust_remote_code=True,
    use_auth_token=True
)

# Tokenization with left padding to optimize memry usage during training
tokenizer = AutoTokenizer.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    padding_side= "left",
    add_eos_token=True,
    add_bos_token=True,
    use_fast=False
)

In [ ]:
# Testing the base model with a few sample inputs
%%time
from transformers import set_seed
seed = 42
set_seed(seed)

index = 10
prompt = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

formated_prompt = f"Instruct: Summarize the following conversation .\n{prompt}\nOutput:\n"
res = gen(original_model, formated_prompt, 100,)
#print(res[0])
output = res[0].split('Output:\n')[1]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{formated_prompt}\n')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

In [ ]:
# Helper functions for formating the input dataset

# converting the dialog-summary pairs into explicit instructions for the LLM
def create_prompt_formats(sample):
    """
    Input is a dictipnary containing dialogue and a summary
    This function reformats the dictonary dataset and concatenates it into prompt format
    """
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruct: Summarize the below conversation."
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"

    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"{sample['dialogue']}" if sample["dialogue"] else None
    response = f"{RESPONSE_KEY}\n{sample['summary']}"
    end = f"{END_KEY}"

    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt

    return sample

from functools import partial

# function that processes the prompts into tokenized ones
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max length: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length

# converts text into tokenized representations
def preprocess_batch(batch, tokenizer, max_length):
    # Tokenizing a batch
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True
    )

def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset):
    "Format the dataset and tokenize it so it is reeady for training"
    "Tokenizer parameter is model tokenizer, param max_length is the max number of tokens to emit fro tokenizer"
    print("Preprocessing dataset")
    dataset = dataset.map(create_prompt_formats)

    # Applying preprocessing to each batch of the dataset and remove unwanted fields
    _preprocessing_function = partial(
        preprocess_batch,
        max_length=max_length,
        tokenizer=tokenizer
        )
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=['id', 'topic', 'dialogue', 'summary']       # deletes original columns that are no logner needed
        )
    
    # filter out the samples that have input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length) 

    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)
    return dataset


# Pre-process dataset
max_length = get_max_length(ORIGINAL_MODEL)
print(max_length)

train_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset['train'])
eval_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset['validation'])

In [ ]:
# PEFT using prepare_model_for_kbit_training method
from peft import LoraConfig, get_peft_model, prepare_model, prepare_model_for_kbit_training

original_model = prepare_model_for_kbit_training(original_model)

config = LoraConfig(
    r=32,   # rank
    lora_alpha=32,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense'
    ],
    bias="none",
    lora_dropout=0.05, 
    task_type="CAUSAL_LM"
)


# enable gradient checkpoint to reduce memory usage during fine tuning
original_model.gradinet_checkpointing_enable()
peft_model = get_peft_model(original_model, config)

print(print_number_of_trainable_model_parameters(peft_model))





In [ ]:
# Training PEFT adapter

output_dir = f"./peft-dialogue-summary-training-{str(int(time.time()))}"

import transformers

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=1000,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=25,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=25,
    evaluation_strategy="steps",
    eval_steps=25,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
)

peft_model.config.use_cache = False

peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

peft_trainer.train()

In [ ]:
import torchfrom transformers import AutoTokenizer, AutoModelForCausalLM

base_model_id = "microsoft/phi-2"
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map='auto',
    quantization_config=bnb_config,
    trust_remote_code=True,
    use_auth_token=True
)

eva_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id, 
    add_bos_token=True,
    trust_remote_code=True,
    use_fast=False
)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "/kaggle/working/peft-dialogue-summary-training-1705417060/checkpoint-1000",torch_dtype=torch.float16,is_trainable=False)

In [ ]:
%%time
from transformers import set_seed
set_seed(seed)

index = 5
dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"Instruct: Summarize the following conversation.\n{dialogue}\nOutput:\n"

peft_model_res = gen(ft_model,prompt,100,)
peft_model_output = peft_model_res[0].split('Output:\n')[1]
#print(peft_model_output)
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

In [ ]:
original_model = AutoModelForCausalLM.from_pretrained(base_model_id, 
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)
import pandas as pd

dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    human_baseline_text_output = human_baseline_summaries[idx]
    prompt = f"Instruct: Summarize the following conversation.\n{dialogue}\nOutput:\n"
    
    original_model_res = gen(original_model,prompt,100,)
    original_model_text_output = original_model_res[0].split('Output:\n')[1]
    
    peft_model_res = gen(ft_model,prompt,100,)
    peft_model_output = peft_model_res[0].split('Output:\n')[1]
    print(peft_model_output)
    peft_model_text_output, success, result = peft_model_output.partition('###')

    original_model_summaries.append(original_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, peft_model_summaries))
 
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'peft_model_summaries'])
df

import evaluate

rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('PEFT MODEL:')
print(peft_model_results)

print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')